# #设计自己的句子生成器

In [1]:
import random

In [2]:
coach = """
coach = 号码 位置 自己 想法  人称 动作 结尾 
号码 = 数字 号
数字 = 单个数字
单个数字 =  1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 
位置 = 前锋， |中场， | 后腰， | 后卫， |门将， 
自己 = 我
人称 = 你 | 你们 | 所有人
想法 = 让 | 要 | 想要 
动作 = 拦截 | 铲球 | 射门 | 回防 | 往死里铲
结尾 = 啊！
"""
player = """
player = 答复 自己 想法 动作 结尾
答复 = 好的， | OK，| 没问题， 
自己 = 我 | 我们 
想法 = 正在 | 准备 | 已经
动作 = 拦截 | 铲球 | 射门 | 回防 | 往死里铲 | 防守
结尾 = 啦！
"""


In [3]:
def create_grammar(gram,split='=',line_split='\n'):
    new_gram={}
    for i in gram.split(line_split):
        if not i.strip():continue
        exp,stmt = i.split(split)
        new_gram[exp.strip()] = [s.split() for s in stmt.split('|')]
    return new_gram

In [4]:
choice = random.choice
def generate_gram(gram,target):
    if target not in gram: return target  #如果target为终结符
    extent = [generate_gram(gram,t) for t in choice(gram[target])]
    return ''.join(e for e in extent)

In [5]:
for n in range(10):
    print (generate_gram(create_grammar(coach),target='coach'))
    print ('-'*2 + generate_gram(create_grammar(player),target='player'))

8号门将，我要你们拦截啊！
--OK，我准备防守啦！
1号后腰，我要你们拦截啊！
--好的，我准备往死里铲啦！
11号后腰，我让所有人拦截啊！
--OK，我正在防守啦！
5号门将，我想要所有人射门啊！
--没问题，我准备射门啦！
6号后腰，我想要你拦截啊！
--好的，我已经铲球啦！
6号门将，我让所有人往死里铲啊！
--OK，我们已经防守啦！
5号中场，我想要你往死里铲啊！
--OK，我准备铲球啦！
10号后卫，我让你回防啊！
--没问题，我们已经拦截啦！
5号前锋，我要你回防啊！
--好的，我准备铲球啦！
9号后卫，我要所有人铲球啊！
--好的，我准备回防啦！


# #Language Model
使用保险行业和豆瓣评论两个个数据集进行语言模型训练

In [210]:
import pandas as pd
import string
import re
import jieba
from collections import Counter

In [192]:
#导入train.txt
txt_filename = 'd:/train.txt'
table = pd.read_table(txt_filename,encoding='utf-8',header =None,sep='\t')
article_tmp= table[0].values
article_txt=[]
for a in article_tmp:
    article_txt.append(a)
len(article_txt)    

c:\users\13569\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until


12889

In [199]:
#导入豆瓣评论的csv
filename = 'd:/movie_comments.csv'
content = pd.read_csv(filename,encoding='utf-8')
content.head()
articles = content['comment'].tolist()
len(articles)

261497

In [201]:
#train.txt+豆瓣csv
articles += article_txt
len(articles)

274386

In [242]:
#正则表达式整理数据
def token(string):
    
    return re.findall(r'^[^\d]\w+',string)

articles_clean =  [''.join(token(str(a))) for a in articles]
len(articles_clean)

274386

In [243]:
with open('Assignment_lesson01_test.txt','w',encoding='utf-8') as f:
    for a in articles_clean:
        f.write(a)

In [244]:
def cut(string): return list(jieba.cut(string))

TOKEN=[]
for i,line in enumerate((open('Assignment_lesson01_test.txt',encoding='utf-8'))):
    #if i % 100 == 0: print(i)
    
    if i > 260000: break #尝试取20+万数据
    
    TOKEN += cut(line)
len(TOKEN)

1139760

In [245]:
words_count=Counter(TOKEN)
words_count.most_common(100) ##正则表达式没写好，还是出现了其他符号‘“’

[('的', 89544),
 ('了', 26932),
 ('是', 17628),
 ('我', 15442),
 ('看', 13963),
 ('电影', 13087),
 ('很', 11090),
 ('好', 7634),
 ('啊', 7517),
 ('都', 7450),
 ('不', 7290),
 ('在', 6801),
 ('和', 6762),
 ('有', 6746),
 ('喜欢', 5565),
 ('就', 5432),
 ('这', 5362),
 ('人', 5136),
 ('一个', 4672),
 ('还', 4653),
 ('故事', 4450),
 ('不错', 4297),
 ('一部', 4242),
 ('好看', 4016),
 ('也', 3858),
 ('片子', 3848),
 ('太', 3779),
 ('你', 3623),
 ('还是', 3478),
 ('剧情', 3425),
 ('没有', 3309),
 ('这个', 3223),
 ('就是', 3205),
 ('吧', 3019),
 ('说', 3007),
 ('没', 2996),
 ('给', 2995),
 ('最', 2885),
 ('片', 2762),
 ('什么', 2737),
 ('又', 2536),
 ('这部', 2512),
 ('得', 2493),
 ('挺', 2452),
 ('拍', 2448),
 ('觉得', 2448),
 ('上', 2421),
 ('经典', 2415),
 ('可以', 2367),
 ('真的', 2285),
 ('让', 2279),
 ('不是', 2243),
 ('想', 2194),
 ('有点', 2181),
 ('时候', 2171),
 ('到', 2139),
 ('这么', 2119),
 ('中', 2104),
 ('被', 2093),
 ('对', 2088),
 ('感觉', 2072),
 ('能', 2068),
 ('导演', 2054),
 ('多', 2054),
 ('看过', 2051),
 ('小', 2051),
 ('这种', 2011),
 ('其实', 1951),
 ('真是', 1890)

In [231]:
TOKEN_2 = [str(t) for t in TOKEN]
TOKEN_2_grammar = [''.join(TOKEN_2[i:i+2]) for i in range(len(TOKEN_2)-2)]

words_count2 = Counter(TOKEN_2_grammar)
words_count2.most_common(100)

[('的电影', 4098),
 ('看的', 3748),
 ('看了', 2067),
 ('的片子', 1737),
 ('的故事', 1682),
 ('的时候', 1488),
 ('看完', 1407),
 ('都是', 1327),
 ('的一部', 1288),
 ('我的', 1076),
 ('的人', 1007),
 ('这部电影', 1003),
 ('好的', 929),
 ('还不错', 911),
 ('很好', 868),
 ('喜欢的', 845),
 ('不错的', 812),
 ('的是', 806),
 ('拍的', 797),
 ('让我', 797),
 ('让人', 795),
 ('这样的', 786),
 ('这是', 781),
 ('电影的', 771),
 ('好看的', 760),
 ('不知道', 748),
 ('不喜欢', 734),
 ('了我', 723),
 ('我觉得', 700),
 ('很喜欢', 683),
 ('中的', 677),
 ('是我', 641),
 ('的我', 622),
 ('的感觉', 617),
 ('最好的', 607),
 ('人的', 597),
 ('也是', 578),
 ('还可以', 565),
 ('是个', 563),
 ('很不错', 556),
 ('的剧情', 538),
 ('的好', 534),
 ('去看', 515),
 ('很有', 508),
 ('我是', 505),
 ('又是', 489),
 ('三星半', 488),
 ('的很', 487),
 ('一部电影', 481),
 ('最喜欢', 481),
 ('这片子', 478),
 ('的作品', 465),
 ('很好看', 454),
 ('我喜欢', 453),
 ('不好看', 440),
 ('是一部', 429),
 ('了吧', 418),
 ('牛逼', 418),
 ('的演技', 413),
 ('看过的', 404),
 ('的经典', 402),
 ('挺好', 398),
 ('的青春', 397),
 ('我就', 396),
 ('自己的', 389),
 ('我看', 388),
 ('经典的', 386),
 ('我也', 384)

In [250]:
def prob_1(w):return words_count[w] / len(TOKEN)
def prob_2(w1,w2):
    if w1 + w2 in words_count2: return words_count2[w1+w2] / len(TOKEN_2_grammar)
    else:
        return 1 / len(TOKEN_2_grammar)

def get_probability(sentence):
    words = cut(sentence)
    sentence_prob = 1
    
    for i,word in enumerate(words[:-1]):
        nextword = words[i+1]
        probability = prob_2(word,nextword)
        sentence_prob *= probability
    return sentence_prob

compared = [
    "隔壁老王昨天做飞机 戈壁老王昨天坐飞机",
    "今天我要去提足求 今天我要去踢足球",
    "开课吧越来越好 越来越豪开课吧",
    "他被公司开除了 它被公司开出了",
    "我想去看电影 我相去砍电影"
]

for w in compared:
    w1,w2 = w.split()
    p1 = get_probability(w1)
    p2 = get_probability(w2)
    
    better = w1 if p1>p2 else w2
    
    
    print('{} is more possible'.format(better))
    print('-'*2 + ' {} with probility {}'.format(w1, p1))
    print('-'*2 + ' {} with probility {}'.format(w2, p2))

戈壁老王昨天坐飞机 is more possible
-- 隔壁老王昨天做飞机 with probility 5.333249638442835e-24
-- 戈壁老王昨天坐飞机 with probility 6.754015490458143e-19
今天我要去踢足球 is more possible
-- 今天我要去提足求 with probility 1.1851665863206301e-24
-- 今天我要去踢足球 with probility 1.3508030980916286e-18
开课吧越来越好 is more possible
-- 开课吧越来越好 with probility 1.6209637177099544e-17
-- 越来越豪开课吧 with probility 6.754015490458143e-19
它被公司开出了 is more possible
-- 他被公司开除了 with probility 1.1851665863206301e-24
-- 它被公司开出了 with probility 5.333249638442835e-24
我想去看电影 is more possible
-- 我想去看电影 with probility 9.6554804761757e-16
-- 我相去砍电影 with probility 6.754015490458143e-19
